## 5.1 학습 모델 저장하고 재사용하기 ##

앞의 포유류와 조류를 구분하는 신경망 모델 재사용

코드 안에 데이터를 같이 넣는 것이 비효율적이므로, 데이터를 csv 파일로 분리한 뒤 해당 파일을 읽어 들여 사용하는 방법 사용

아래 내용을 data.csv 파일로 저장

<pre>
# 털, 날개, 기타, 포유류, 조류
    0, 0, 1, 0, 0
    1, 0, 0, 1, 0
    1, 1, 0, 0, 1
    0, 0, 1, 0, 0
    0, 0, 1, 0, 0
    0, 1, 0, 0, 1
<code>

이때 저장시, 한글이 있으면 읽는데 문제 발생가능 -> 한글 주석 제외 입력

1열과 2열 : 털과 날개 -> feature 의미

3열 -  마지막 : 개체의 종류 나타내는 데이터 -> one-hot encoding 사용


In [2]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter = ',', unpack = True, dtype = 'float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

데이터를 읽어 들이고 변환하는 코드로 프로그램 시작

1. numpy 라이브러리의 loadtxt 함수를 이용하여 간단하게 데이터를 읽어 들임
2. 1열과 2열은 x_data로 변환
3. 3열부터 마지막 열까지는 y_data로 변환

#### loadtxt 의 unpack / transpose ####
loadtxt의 unpack 매개변수와 transpose 함수는 데이터 구조를 변환시킴

    원시 데이터            unpack = True
[[0. 0. 1. 0. 0.]       [[0. 1. 1. 0. 0. 0.]

[1. 0. 0. 1. 0.]        [0. 0. 1. 0. 0. 1.]

[1. 1. 0. 0. 1.]   ->   [1. 0. 0. 1. 1. 0.]

[0. 0. 1. 0. 0.]        [0. 1. 0. 0. 0. 0.]

[0. 0. 1. 0. 0.]        [0. 0. 1. 0. 0. 1.]]

[0. 1. 0. 0. 1.]]

    data[0:2]              np.transpose

[[0. 1. 1. 0. 0. 0.]   ->   [[0. 0.] 

[0. 0. 1. 0. 0. 1.]]        [1. 0.]


[1. 1.]

[0. 0.]

[0. 0.]

[0. 1.]]

* 읽어 들이거나 잘라낸 데이터의 행과 열을 뒤바꿔주는 옵션과 함수

-> 머신러닝, 딥러닝에서는 학습 알고리즘을 적용하거나 행렬 연산을 효율적으로 하기 위해 데이터의 행과 열을 상황에 맞게 변환하는 일이 잦음

In [3]:
global_step = tf.Variable(0, trainable = False, name = 'global_step')

모델을 저장할 떄 쓸 변수를 하나 만듦 

-> 학습에 직접 사용되지 않고, 학습 횟수를 카운트하는 변수

=> 이를 위해 **trainable = False** 옵션을 걸어줌

In [4]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10,20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3  = tf.Variable(tf.random_uniform([20,3], -1., 1.))
model = tf.matmul(L2, W3)

cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = Y, logits = model))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
train_op = optimizer.minimize(cost, global_step = global_step)

계층을 이전 것에서 하나 더 늘리고, bias 없이 weight만 사용

W2의 형태가 [10,20]인 것을 확인 할 수 있음!!! <= 뒷 단의 입력 크기가 20이기 때문! 

**Note**
신경망의 layer수와 hidden layer의 unit수를 늘리면 복잡도가 높은 문제 해결에 도움이 됨

그러나 무조건적인 도움 No / Overfitting의 문제 가능! 

-> Neural Network에서 hidden layer 수와 unit 수를 최적화 하는 것이 중요함


* 마지막 줄의 global_step 매개변수에 대해 global_step 변수를 넘겨주었음 

-> 최적화함수가 변수를 최적화할때마다 global_step 변수의 값을 1씩 증가

In [5]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

모델 구성 완료 -> 세션을 열고 최적화 실행

tf.global_varaibles : 앞서 정의한 변수들을 가져오는 함수

-> 앞서 정의한 벼수들을 모두 가져와서 이 변수들을 파일에 저장하거나 이전에 학습한 결과를 불러와 담는 변수들로 사용!

In [9]:
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

./model directory에 기존에 학습해둔 모델이 있는지를 확인

-> 모델이 있다면 saver.store 함수를 이용해 학습된 값들을 불러옴

-> 없다면 변수를 새로 초기화

=> **학습된 모델을 저장한 파일을 checkpoint file**이라 함 

In [10]:
for step in range(2):
    sess.run(train_op, feed_dict = {X: x_data, Y: y_data})
    
    print('Step: %d, ' % sess.run(global_step), 
         'Cost: %.3f' %sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Step: 1,  Cost: 1.104
Step: 2,  Cost: 1.046


최적화 수행

step이 아닌 global_step을 이용해 학습을 몇 번째 진행하고 있는지 확인

global_step은 tensor type으 변수이므로 값을 가져올 때 sess.run(global_step)을 이용

-> 학습시킨 모델을 저장한 후 불러들여서 재학습한 결과를 보기 위해 학습 횟수 2번으로 설정

In [11]:
saver.save(sess, './model/dnn.ckpt', global_step = global_step)

'./model/dnn.ckpt-2'

위의 코드 중 두 번째 매개변수는 체크포인트 파일의 위치와 이름

-> 상위 directory (./model)은 미리 생성되어 있어야 함

* global_step의 값은 저장할 파일의 이름에 추가로 붙게 됨 <- 텐서 변수 또는 숫자값을 넣어줄 수 있음!

-> 여러 상태의 체크포인트를 만들 수 있고, 가장 효과적인 체크포인트를 선별해서 사용할 수 있음

In [12]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y,1)
print('예측값: ', sess.run(prediction, feed_dict= {X: x_data}))
print('실제값: ', sess.run(target, feed_dict= {Y: y_data}))

예측값:  [0 1 1 0 0 0]
실제값:  [0 1 2 0 0 2]


In [13]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' %sess.run(accuracy * 100, feed_dict = {X: x_data, Y: y_data}))

정확도: 66.67


In [ ]:
##전체 코드
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter = ',', unpack = True, dtype = 'float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

##################
# 신경망 모델 구성
##################
global_step = tf.Variable(0, trainable = False, name = 'global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X,W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = Y, logits = model))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
train_op = optimizer.minimize(cost, global_step = global_step)

###################
# 신경망 모델 학습
###################
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
for step in range(2):
    sess.run(train_op, feed_dict = {X: x_data, Y: y_data})
    
    print('Step: %d, ' % sess.run(global_step),
         'Cost: %.3f' %sess.run(cost, feed_dict = {X: x_data, Y: y_data}))

saver.save(sess, './model/dnn.ckpt', global_step = global_step)

###########
# 결과확인
###########
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값: ', sess.run(prediction, feed_dict = {X: x_data}))
print('실제값: ', sess.run(target, feed_dict = {Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' %sees.run(accuracy * 100, feed_dict = {X: x_data, Y: y_data}))